## Предсказания свойств ФБ, с помощью CNN (эмбеддинги (ESM C))

### Подготовка к работе

In [ ]:
! pip install torch tqdm
#! pip install --upgrade git+https://github.com/rimgro/biocadprotein.git

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import pandas as pd

from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from fpgen.prop_prediction.dataset import FPbase
from fpgen.prop_prediction.metrics import get_regression_metrics

### Загрузка датасета

In [ ]:
dataset = FPbase('dataset.csv')

In [ ]:
x_train, y_train = dataset.get_train('em_max')
x_test, y_test = dataset.get_test('em_max')

In [ ]:
import pickle

with open('sequence.pickle', 'rb') as file:
    seq = pickle.load(file)

### Подготовка данных (padding эмбеддингов)

In [ ]:
def preprocessing_x(x_tr, x_t):
    matrix_tr = []
    for i in range(len(x_tr)):
        matrix_tr.append(seq[x_tr.iloc[i]])

    matrix_t = []
    for i in range(len(x_t)):
        matrix_t.append(seq[x_t.iloc[i]])

    max_h = max(max(t.shape[1] for t in matrix_tr), max(t.shape[1] for t in matrix_t))
    max_w = max(max(t.shape[2] for t in matrix_tr), max(t.shape[2] for t in matrix_t))

    def pad_tensor_list(tensor_list):
        padded = []
        for t in tensor_list:
            c, h, w = t.shape
            pad_h = max_h - h
            pad_w = max_w - w
            padded_tensor = F.pad(t, (0, pad_w, 0, pad_h))
            padded.append(padded_tensor)
        return torch.stack(padded)

    train = pad_tensor_list(matrix_tr)
    test = pad_tensor_list(matrix_t)
    return train, test

In [ ]:
x_train_p, x_test_p = preprocessing_x(x_train, x_test)

In [ ]:
x_train_p.shape

torch.Size([643, 1, 739, 960])

In [ ]:
y_train_p = torch.tensor(y_train.to_numpy(), dtype=torch.float32).view(-1, 1)  # [643, 1]
y_test_p = torch.tensor(y_test.to_numpy(), dtype=torch.float32).view(-1, 1)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

dataset_train = TensorDataset(x_train_p, y_train_p)
train_loader = DataLoader(dataset_train, batch_size=32, shuffle=True)

dataset_test = TensorDataset(x_test_p, y_test_p)
test_loader = DataLoader(dataset_test, batch_size=32, shuffle=True)

### Архитектура CNN

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, padding=2)
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, padding=2)
        self.fc1 = nn.Linear(32 * 184 * 240, 128) 
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

### Обучение модели

In [ ]:
#device = 'cpu'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = CNN()
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_fn = nn.MSELoss()

for epoch in tqdm(range(100)):
    model.train()
    for xb, yb in train_loader:
        xb = xb.to(device)
        yb = yb.to(device)

        preds = model(xb)
        loss = loss_fn(preds, yb)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch}: loss = {loss.item():.4f}")


cuda


  1%|█▏                                                                                                               | 1/100 [00:03<05:32,  3.36s/it]

Epoch 0: loss = 0.4711


  2%|██▎                                                                                                              | 2/100 [00:06<05:26,  3.33s/it]

Epoch 1: loss = 1.1844


...

Epoch 97: loss = 0.0159


 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 99/100 [05:28<00:03,  3.32s/it]

Epoch 98: loss = 0.0438


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [05:32<00:00,  3.32s/it]

Epoch 99: loss = 0.4560


### Тестирование и метрики

In [ ]:
model = model.to('cpu')
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for xb, yb in test_loader:
        preds = model(xb)
        y_true.extend(yb.cpu().numpy().flatten())
        y_pred.extend(preds.cpu().numpy().flatten())


In [ ]:
y_true_rescaled = dataset.rescale_targets(y_true, 'em_max')
y_pred_rescaled = dataset.rescale_targets(y_pred, 'em_max')

get_regression_metrics(y_pred_rescaled, y_true_rescaled)

{'rmse': 32.17417,
 'mae': 22.368298,
 'r2': 0.7198959653410872,
 'mae_median': 14.933685}